# Evidently Report & TestSuites Customization

In [ ]:
import pandas as pd

from sklearn import datasets
from sklearn import ensemble
from sklearn import model_selection

from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite

from evidently.metrics import DataDriftTable
from evidently.metrics import ClassificationQualityMetric
from evidently.metrics import ColumnQuantileMetric
from evidently.metrics import ColumnValueRangeMetric

from evidently.metric_preset import DataDriftPreset

from evidently.metrics.base_metric import generate_column_metrics

from evidently.tests import TestCorrelationChanges
from evidently.tests import TestShareOfDriftedColumns
from evidently.tests import TestColumnQuantile
from evidently.tests import TestColumnValueMin
from evidently.tests import TestPrecisionScore
from evidently.tests import TestRecallScore

from evidently.test_preset import NoTargetPerformanceTestPreset

from evidently.tests.base_test import generate_column_tests

from evidently.options import ColorOptions

from evidently.tests.utils import approx

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
#Dataset for binary label and probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

In [ ]:
bcancer_ref.head()

## Metric Parameters Setting

In [ ]:
#Dataset metrics with parameters
dataset_report = Report(metrics=[
    ClassificationQualityMetric(probas_threshold=0.5),
    DataDriftTable(num_stattest='kl_div', cat_stattest='psi', stattest_threshold=0.1),  
])

dataset_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
dataset_report

In [ ]:
#Column metrics with parameters
columns_report = Report(metrics=[
    ColumnQuantileMetric(column_name='mean radius', quantile=0.75), 
    ColumnValueRangeMetric(column_name='mean texture', left=10, right=20)
])

columns_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
columns_report

In [ ]:
#Auto-generated column metrics with parameters
metric_generator_report = Report(
    metrics=[
        generate_column_metrics(
            ColumnValueRangeMetric,
            columns=['mean radius', 'mean texture', 'mean perimeter'],
            parameters={"left": 5, "right": 25}
        )
    ]
)
metric_generator_report.run(current_data=bcancer_ref, reference_data=bcancer_cur)
metric_generator_report

In [ ]:
#Metric presets with parameters
preset_report = Report(metrics=[
    DataDriftPreset(columns = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean symmetry'], 
                    drift_share = 0.1, stattest_threshold=0.2),  
])

preset_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
preset_report

In [ ]:
#Metric presets with parameters
preset_report = Report(metrics=[
    DataDriftPreset(columns = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean symmetry'], 
                    per_column_stattest = {
                        'mean radius':'hellinger', 
                        'mean texture':'t_test', 
                        'mean perimeter':'mannw', 
                        'mean area':'wasserstein', 
                        'mean symmetry':'cramer_von_mises'
                    }),  
])

preset_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
preset_report

## Test Parameters and Conditions Setting

In [ ]:
#Dataset tests with parameters and conditions
dataset_suite = TestSuite(tests=[
    TestCorrelationChanges(method='spearman', corr_diff=0.5),
    TestShareOfDriftedColumns(gt=0)
])

dataset_suite.run(current_data=bcancer_ref, reference_data=bcancer_cur)
dataset_suite

In [ ]:
#Column tests with parameters and conditions
column_tests_suite = TestSuite(tests=[
    TestColumnQuantile(column_name='mean perimeter', quantile=0.25, eq=approx(70, relative=0.08)),
    TestPrecisionScore(probas_threshold=0.8, gt=0.99),
    TestRecallScore(probas_threshold=0.8, gt=0.99)
])

column_tests_suite.run(current_data=bcancer_ref, reference_data=bcancer_cur)
column_tests_suite

In [ ]:
#Auto-generated column tests with parameters and conditions
test_generator_suite = TestSuite(tests=[
    generate_column_tests(TestColumnValueMin, columns="num")
])

test_generator_suite.run(current_data=bcancer_ref, reference_data=bcancer_cur)
test_generator_suite

In [ ]:
#Auto-generated column tests with parameters and conditions
test_generator_suite = TestSuite(tests=[
    generate_column_tests(TestColumnValueMin, columns=[
        'mean radius', 
        'mean texture', 
        'mean perimeter', 
        'mean area', 
        'mean symmetry'
    ], parameters={"gt": 0})
])

test_generator_suite.run(current_data=bcancer_ref, reference_data=bcancer_cur)
test_generator_suite

In [ ]:
#Test presets with parameters and conditions
no_target_performance = TestSuite(tests=[
    NoTargetPerformanceTestPreset(columns=['mean radius', 
                                           'mean texture', 
                                           'mean perimeter', 
                                           'mean area', 
                                           'mean symmetry'
                                          ])
])

no_target_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
no_target_performance

## Colors Specification 

In [ ]:
#options
color_scheme = ColorOptions()
color_scheme.primary_color = "#5a86ad"
color_scheme.fill_color = "#fff4f2"
color_scheme.zero_line_color = "#016795"
color_scheme.current_data_color = "#c292a1" 
color_scheme.reference_data_color = "#017b92"

In [ ]:
test_generator_suite = TestSuite(tests=[
    generate_column_tests(TestColumnValueMin, columns=[
        'mean radius', 
        'mean texture', 
        'mean perimeter', 
        'mean area', 
        'mean symmetry'
    ], parameters={"gt": 0})
],
                                options=[color_scheme])

test_generator_suite.run(current_data=bcancer_ref, reference_data=bcancer_cur)
test_generator_suite

In [ ]:
columns_report = Report(metrics=[
    ColumnQuantileMetric(column_name='mean radius', quantile=0.75), 
    ColumnValueRangeMetric(column_name='mean texture', left=10, right=20)
],
                       options=[color_scheme])

columns_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
columns_report